In [211]:
import pandas as pd
import plotly.express as px

In [222]:
data = pd.read_csv('data/indicators/data.csv', index_col=0)

In [223]:
data

,ISO,Year,Indicator,From,Description,Country,Source,URL,DownloadDate,Variable Type,Imputed From,Imputed from Year,Value
832651,PRT,1960,GE1,WB API,CO2 emissions (metric tons per capita),Portugal,"Carbon Dioxide Information Analysis Center, En...",https://api.worldbank.org/v2/country/all/indic...,2020-11-16,Full Indicator,NaN,NaN,0.928578
836616,PRY,1960,GE1,WB API,CO2 emissions (metric tons per capita),Paraguay,"Carbon Dioxide Information Analysis Center, En...",https://api.worldbank.org/v2/country/all/indic...,2020-11-16,Full Indicator,NaN,NaN,0.159855
114986,BLZ,1960,GE1,WB API,CO2 emissions (metric tons per capita),Belize,"Carbon Dioxide Information Analysis Center, En...",https://api.worldbank.org/v2/country/all/indic...,2020-11-16,Full Indicator,NaN,NaN,0.477972
828686,PRK,1960,GE1,WB API,CO2 emissions (metric tons per capita),"Korea, Dem. People’s Rep.","Carbon Dioxide Information Analysis Center, En...",https://api.worldbank.org/v2/country/all/indic...,2020-11-16,Full Indicator,NaN,NaN,2.084804
848511,PST,1960,GE1,WB API,CO2 emissions (metric tons per capita),Post-demographic dividend,"Carbon Dioxide Information Analysis Center, En...",https://api.worldbank.org/v2/country/all/indic...,2020-11-16,Full Indicator,NaN,NaN,7.732650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
765222,NOR,2020,AB2,IMPUTED,[(Proportion of population with access to elec...,NaN,World Bank AND Global Health Observatory (GHO)...,NaN,NaN,Full Indicator,SDG API,2017.0,97.500000
765223,NOR,2020,GB1,SDG API,Proportion of seats held by women in national ...,Norway,"Inter-Parliamentary Union (IPU), the database ...",https://unstats.un.org/SDGAPI/v1/sdg/Series/Data,2020-11-16,Full Indicator,NaN,NaN,41.420000
765224,NOR,2020,SL2,IMPUTED,(Organic agriculture land) / (Total agricultur...,NaN,FAO,NaN,NaN,Full Indicator,MANUAL,2017.0,4.434023
765226,NOR,2020,GV1,IMPUTED,"Adjusted net savings, including particulate em...",NaN,World Bank staff estimates based on sources an...,https://api.worldbank.org/v2/country/all/indic...,NaN,Full Indicator,WB API,2018.0,18.208421


In [213]:
dimension_properties = pd.read_csv('data/indicators/dimension_properties.csv', index_col=0)


In [214]:
dimension_options = dimension_properties.Dimension.values


In [217]:
data['Imputed From'] = data['Imputed From'].fillna('')
data['Imputed from Year'] = data['Imputed from Year'].fillna('')

In [218]:
data['From'] = data['From'] + ' ' + data['Imputed From'] + ' ' +data['Imputed from Year'].astype(str)

In [219]:
indicator_properties = pd.read_csv('data/indicators/indicator_properties.csv', index_col=0)
indicator_properties = indicator_properties

In [220]:
indicator_properties

,Indicator,Description,Unit,Dimension,Impact,minYear,maxYear
NaN,GE1,Ratio CO2 emissions incl LUCF to population,Tons CO2e per Capita,Natural Capital Protection,negative,1960,2020
1.0,GE3,"Ratio non-CO2 emissions (CH4, N2O and F-gas) i...",Tons CO2e per Capita,Natural Capital Protection,negative,1961,2020
2.0,SE1,Palma Ratio,Ratio,Social Inclusion,negative,1967,2020
3.0,ME1,Total domestic material consumption (DMC) per ...,kg per constant 2005 GDP,Efficient and Sustainable Resource Use,negative,1970,2020
4.0,GB3,"Getting paid, laws and regulations for equal g...",Score (0-100),Social Inclusion,positive,1971,2020
5.0,GN1,Share of patent publications in environmental ...,Percent,Green Economic Opportunities,positive,1980,2020
6.0,CV3,Share of terrestrial and marine protected area...,Percent,Natural Capital Protection,positive,1990,2020
7.0,EE2,Renewable energy share in the total final ener...,Percent,Efficient and Sustainable Resource Use,positive,1990,2020
8.0,GE2,"Ratio non-CO2 emissions (CH4, N2O and F-gas) e...",Tons CO2e per Capita,Natural Capital Protection,negative,1990,2020
9.0,EE1,Energy intensity level of primary energy,Megajoules per constant 2011 purchasing power ...,Efficient and Sustainable Resource Use,negative,1990,2020


In [221]:
indicator_properties['display_name'] = indicator_properties['Indicator'] + ': ' + indicator_properties['Description']

In [180]:
data = data[data.Indicator.isin(indicator_properties.Indicator)]

In [181]:
ISO_list = ['DEU', 'FRA', 'KOR']
Indicator = 'EE1'

In [184]:
def indicator_line_charts(data, indicator_properties, ISO_list, Indicator):
    indicator_properties = indicator_properties.set_index('Indicator')

    df = data[(data.ISO.isin(ISO_list)) & (data.Indicator == Indicator)]
    title = indicator_properties.loc[Indicator]['Description']
    xaxis_title = indicator_properties.loc[Indicator]['Unit']
    fig = px.scatter(df,
              x='Year',
              y='Value',
              color='ISO',
              hover_data={'Value': True,
                          'Year':False,
                          'Country': True,
                          'Source': True,
                          'From': True},
              height=400,)
    fig.update_layout(title=title, yaxis_title=xaxis_title)
    fig.update_traces(mode='lines+markers')

    return fig

In [185]:
indicator_line_charts(data, indicator_properties, ISO_list, Indicator)